# Marking tests
Assigning letter grades: array vs procedural programming

Computers might seem like clever tools but they are not very smart. They cannot currently understand [natural languages](https://en.wikipedia.org/wiki/Natural_language) like English. You have to tell a computer what to do using special languages called [programming languages](https://en.wikipedia.org/wiki/Programming_language). There are many [programming languages](https://en.wikipedia.org/wiki/List_of_programming_languages). 

A program is a series of instructions, much like a cooking recipe. In most programming languages, the computer reads the program line-by-line and executes each one at a time. The computer chip itself often reads instructions [one at a time](https://en.wikipedia.org/wiki/Turing_machine), but these days it can often [do many](https://en.wikipedia.org/wiki/Parallel_computing) things [at once](https://en.wikipedia.org/wiki/Vector_processor).

In this example, we will write a program to assign letter grades to students' test scores. We will start by turning the problem description into pseudocode to express the program before turning it into real code with APL. 

*Problem:* Assign letter grades to students' test scores according to the table. 

| Minimum score | Grade |
|---------------|-------|
|80             | A     |
|70             | B     |
|60             | C     |
|50             | D     |
| 0             | F     |

If a human were marking grades, they might take one scored paper at a time and decide on which letter grade to write by reading each score.

*Procedural psuedocode:*
```C
scores = [93,85,45,10,70,16,93,63,41,7,95,45,76] 
For each score in scores:
  If score is greater than 80:
    Write "A"
  Else If score is greater than 70:
    Write "B"
  Else If score is greater than 60:
    Write "C"
  Else If score is greater than 50:
    Write "D"
  Else
    Write "F"
```

So-called [procedural programming languages](https://en.wikipedia.org/wiki/Procedural_programming) will have actualy code that somewhat closely resembles this pseudocode. APL has a much more mathematical flavour to it. Let's see what APL does when we tackle this problem: 

In [3]:
scores←93 85 45 10 70 16 93 63 41 7 95 45 76
scores>80
scores>70
scores>60
scores>50
scores>0

1 1 0 0 0 0 1 0 0 0 1 0 0

1 1 0 0 0 0 1 0 0 0 1 0 1

1 1 0 0 1 0 1 1 0 0 1 0 1

1 1 0 0 1 0 1 1 0 0 1 0 1

1 1 1 1 1 1 1 1 1 1 1 1 1

Above you can see that APL compares all of the `scores` to each grade. This is done by [scalar extension](Scalar.ipynb). What if we could compare all of the scores to all of the grade boundaries at once?

In [9]:
scores∘.>80 70 60 50 0

1 1 1 1 1
1 1 1 1 1
0 0 0 0 1
0 0 0 0 1
0 0 1 1 1
0 0 0 0 1
1 1 1 1 1
0 0 1 1 1
0 0 0 0 1
0 0 0 0 1
1 1 1 1 1
0 0 0 0 1
0 1 1 1 1

We just used an [outer product]() to apply the `>` greater-than function to every combination of score and grade boundary. The resulting [matrix](Arrays.ipynb) has a shape `13 5`. It has `⍴scores` columns and `⍴80 70 60 50 0` rows.

In [10]:
⍴scores
⍴80 70 60 50 0
⍴scores∘.>80 70 60 50 0

13

5

13 5

If we transpose the array it looks like our previous procedural example.

In [11]:
⍉scores∘.>80 70 60 50 0

1 1 0 0 0 0 1 0 0 0 1 0 0
1 1 0 0 0 0 1 0 0 0 1 0 1
1 1 0 0 1 0 1 1 0 0 1 0 1
1 1 0 0 1 0 1 1 0 0 1 0 1
1 1 1 1 1 1 1 1 1 1 1 1 1

Now we want to generate an vector which is a translation of the numeric scores into letter grades.

In [17]:
scores
'ABCDF'[1 1 5 5 3 5 1 3 5 5 1 5 2]

93 85 45 10 70 16 93 63 41 7 95 45 76

AAFFCFACFFAFB

Clearly the simple way is to get each of the letter grades from a character vector using [square bracket indexing](). We just need to generate the indices (the numbers inside the square brackets).

In [18]:
+/scores∘.>80 70 60 50 0

5 5 1 1 3 1 5 3 1 1 5 1 4

There is a vector with numbers between `1` and `5`, but they're clearly not the right numbers...

In [22]:
1+5-+/scores∘.>80 70 60 50 0 ⍝ This works but is starting to look ugly...

1 1 5 5 3 5 1 3 5 5 1 5 2

In [24]:
'FDCBA'[+/scores∘.>80 70 60 50 0] ⍝ Just flip the letters around!

AAFFCFACFFAFB

### Control structures
Now it is actually possible in Dyalog to directly translate our pseudocode from earlier into a program using [control structures](). However, this tends to be **slow APL** and is generally **considered bad**.

In [29]:
∇ grades←LetterGrade scores
  grades←'' ⍝ Initialise empty character vector
  :For score :In scores
    :If score > 80
      grades,←'A'
    :ElseIf score > 70
      grades,←'B'
    :ElseIf score > 60
      grades,←'C'
    :ElseIf score > 50
      grades,←'D'
    :Else
      grades,←'F'
    :EndIf
  :EndFor
∇

In [30]:
⎕←LetterGrade scores

AAFFCFACFFAFB

To end this tutorial, let's compare the speed of the two approaches.

In [32]:
]Runtime -c "'FDCBA'[+/scores∘.>80 70 60 50 0]" "LetterGrade scores"

'FDCBA'[+/scores∘.>80 70 60 50 0] → 1.7E¯6 | 0% ⎕⎕⎕ 
 LetterGrade scores → 2.1E¯5 | +1133% ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

 
 
 ⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕⎕

The method using control structures is over 1000% slower. Use control structures sparingly in APL.